# INTRODUCTION

In [1]:

# Import SparkSession
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("STREAMING_DWH") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()


In [2]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
df = spark \
    .readStream \
    .format("json") \
    .load("data/product/")

df.printSchema()

root
 |-- category: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- contains_caffeine: boolean (nullable = true)
 |-- contains_fruit: boolean (nullable = true)
 |-- contains_nuts: boolean (nullable = true)
 |-- contains_veggies: boolean (nullable = true)
 |-- event_time: string (nullable = true)
 |-- item: string (nullable = true)
 |-- price: double (nullable = true)
 |-- product_id: string (nullable = true)
 |-- size: string (nullable = true)



In [3]:
from pyspark.sql.functions import current_timestamp, input_file_name

def create_bronze_streaming_table(source, target):
    # Generates a source path based on table name, reads all files from that and inserts into bronze schema

    query = (
        spark.readStream
        .format("json")
        .load(source)
        .withColumn("meta_ingestion_ts", current_timestamp())
        .withColumn("meta_filename", input_file_name())
        .writeStream
        .outputMode("append")
        .format("delta")
        .option("checkpointLocation", f"spark-warehouse/_checkpoints/{target}")
        .toTable(target)
    )
    return query

query1 = create_bronze_streaming_table(source="data/inventory", target="bronze_inventory")
query2 = create_bronze_streaming_table(source="data/product", target="bronze_product")
query3 = create_bronze_streaming_table(source="data/purchase", target="bronze_purchase")

# Use the code 
# spark.streams.awaitAnyTermination()


In [4]:
spark.sql("SELECT * FROM bronze_product").show(10)

+--------------------+----+-----------------+--------------+-------------+----------------+--------------------+---------------+-----+----------+------+--------------------+--------------------+
|            category|cogs|contains_caffeine|contains_fruit|contains_nuts|contains_veggies|          event_time|           item|price|product_id|  size|   meta_ingestion_ts|       meta_filename|
+--------------------+----+-----------------+--------------+-------------+----------------+--------------------+---------------+-----+----------+------+--------------------+--------------------+
|Superfoods Smoothies| 2.1|            false|          true|        false|            true|2024-02-17 19:27:...|Get Up and Goji| 5.99|      SF06|24 oz.|2024-02-21 17:49:...|file:///home/pete...|
|Superfoods Smoothies| 2.1|            false|          true|        false|            true|2024-02-17 19:32:...|Get Up and Goji| 5.99|      SF06|24 oz.|2024-02-21 17:49:...|file:///home/pete...|
|Superfoods Smoothies| 2.

# SILVER TABLES: SLOWLY CHANGING DIMENSIONS (SCD) - TYPE 1 & 2

In [7]:
from pyspark.sql.functions import md5, concat_ws, lit, row_number, monotonically_increasing_id
from pyspark.sql.types import BooleanType, TimestampType
from pyspark.sql.window import Window
from utils import reorder_columns_in_dataframe

def create_silver_scd1_table(
    source : str, 
    target : str,
    timestamp_key : str,
    surrogate_key : str
):

    # Load data and calculate hashdiff string based on all columns that doesn't contain "meta_" in the name
    df = spark.sql(f"select * from {source} order by {timestamp_key}")
    df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))

    # Set default values for meta_last_updated
    df = df.withColumn("meta_last_updated", current_timestamp())

    # Generate surrogate key
    df = df.withColumn(surrogate_key, monotonically_increasing_id())

    # Reorder columns
    df = reorder_columns_in_dataframe(df=df, 
                                      columns_to_front=[surrogate_key],
                                      columns_to_back=[c for c in df.columns if "meta_" in c],
                                      columns_to_delete=["meta_filename"])
    
    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)
    spark.sql(f"CREATE TABLE IF NOT EXISTS {target} LIKE {tmp_view_name} USING DELTA")

    # Merge into target table 
    merge_query = f"""
        MERGE INTO {target} AS target
        USING {tmp_view_name} AS source ON target.{surrogate_key} = source.{surrogate_key}
        WHEN MATCHED AND target.meta_hashdiff <> source.meta_hashdiff THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """
    spark.sql(merge_query).show()


In [9]:

create_silver_scd1_table (
    source="bronze_purchase",
    target="silver_purchase_scd1",
    timestamp_key="transaction_time",
    surrogate_key="transaction_sid",
)

create_silver_scd1_table (
    source="bronze_inventory",
    target="silver_inventory_scd1",
    timestamp_key="event_time",
    surrogate_key="inventory_sid",
)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|              139|               0|               0|              139|
+-----------------+----------------+----------------+-----------------+

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|               35|               0|               0|               35|
+-----------------+----------------+----------------+-----------------+



In [13]:
def create_silver_scd2_table(source, target, merge_key, timestamp_key, surrogate_key):

    # Load data and calculate hashdiff string based on all columns that doesn't contain "meta_" in the name
    df = spark.sql(f"select * from {source}")
    df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))

    # Set default values for meta columns
    df = df.withColumn("meta_is_current", lit(1).cast(BooleanType()))
    df = df.withColumn("meta_valid_from", df[timestamp_key])
    df = df.withColumn("meta_valid_to", lit('9999-12-31').cast(TimestampType()))

    # Calculate surrogate key
    df = df.withColumn(surrogate_key, monotonically_increasing_id())

    # Calculate sequence numbers if source data contain multiple rows for each merge_key
    window_spec = Window.partitionBy(merge_key).orderBy(timestamp_key)
    df = df.withColumn("meta_sequence", row_number().over(window_spec))

    # Reorder columns in dataframe
    df = reorder_columns_in_dataframe(
        df=df, 
        columns_to_front=[surrogate_key, merge_key],
        columns_to_back=[c for c in df.columns if "meta_" in c],
        columns_to_delete=["meta_filename"]
    )

    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)
    spark.sql(f"CREATE TABLE IF NOT EXISTS {target} LIKE {tmp_view_name} USING DELTA")

    # Get list of sequences
    lst_sequence = sorted([p.meta_sequence for p in df.select('meta_sequence').distinct().collect()])

    # Run SCD2 table 
    for seq_num in lst_sequence:
        merge_query = f"""
            MERGE INTO {target} AS target
            USING (
                SELECT * FROM {tmp_view_name}
                WHERE meta_sequence = {seq_num}
            ) AS source ON target.{merge_key} = source.{merge_key}
            WHEN MATCHED AND target.meta_is_current = true AND target.meta_hashdiff <> source.meta_hashdiff
                THEN UPDATE SET meta_is_current = false, meta_valid_to = source.{timestamp_key}
            WHEN NOT MATCHED 
                THEN INSERT *
        """
        spark.sql(merge_query).show()

        insert_query = f"""
            INSERT INTO {target}
            SELECT * FROM 
            (
                SELECT source.* 
                FROM {tmp_view_name} source
                JOIN {target} target ON target.{merge_key} = source.{merge_key}
                WHERE source.meta_sequence = {seq_num}
                AND target.meta_hashdiff <> source.meta_hashdiff 
            )
        """
        spark.sql(insert_query)

In [12]:
# Create SCD2 tables
create_silver_scd2_table(
    source = "bronze_product",
    target = "silver_product_scd2",
    merge_key = "product_id",
    timestamp_key = "event_time",
    surrogate_key = "product_sid"
)

NameError: name 'silver_scd2_table' is not defined

In [ ]:
spark.sql("select * from silver_purchase_scd1").show(5)
spark.sql("select * from silver_inventory_scd1").show(5)
spark.sql("select * from silver_product_scd2 order by product_sid").show(5)

In [ ]:

# def create_gold_dimension_table():


In [10]:
from utils import generate_dim_table_references

def create_gold_fact_table(
    source : str, 
    target : str,
    surrogate_key : str,
    timestamp_key : str,
    dim_table_refs : dict
):

    # Generate and run SQL query
    df = spark.sql(generate_dim_table_references(source, timestamp_key, dim_table_refs))

    # Reorder columns in dataframe
    df = reorder_columns_in_dataframe(
        df=df, 
        columns_to_front=[surrogate_key] + [r["surrogate_key"] for r in dim_table_refs],
        columns_to_back=[c for c in df.columns if "meta_" in c]
    )

    # Create an empty Delta table with the same schema
    tmp_view_name = "temporaryView"
    df.createOrReplaceTempView(tmp_view_name)
    spark.sql(f"CREATE TABLE IF NOT EXISTS {target} LIKE {tmp_view_name} USING DELTA")

    # Merge into target table 
    merge_query = f"""
        MERGE INTO {target} AS target
        USING {tmp_view_name} AS source ON target.{surrogate_key} = source.{surrogate_key}
        WHEN MATCHED AND target.meta_hashdiff <> source.meta_hashdiff THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """

    spark.sql(merge_query).show()
    

In [11]:
create_gold_fact_table (
    source="silver_purchase_scd1",
    target="gold_fact_purchase",
    surrogate_key="transaction_sid",
    timestamp_key="transaction_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)

create_gold_fact_table (
    source="silver_inventory_scd1",
    target="gold_fact_inventory",
    surrogate_key="inventory_sid",
    timestamp_key="event_time",
    dim_table_refs=[{"table_name": "silver_product_scd2", "merge_key": "product_id", "surrogate_key": "product_sid"}]
)


SELECT src.*, silver_product_scd2.product_sid 
FROM silver_purchase_scd1 src
LEFT JOIN silver_product_scd2 ON silver_product_scd2.product_id = src.product_id
        AND src.transaction_time BETWEEN silver_product_scd2.meta_valid_from AND silver_product_scd2.meta_valid_to


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `silver_product_scd2` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 10;
'Project [src.*, 'silver_product_scd2.product_sid]
+- 'Join LeftOuter, (('silver_product_scd2.product_id = 'src.product_id) AND (('src.transaction_time >= 'silver_product_scd2.meta_valid_from) AND ('src.transaction_time <= 'silver_product_scd2.meta_valid_to)))
   :- SubqueryAlias src
   :  +- SubqueryAlias spark_catalog.default.silver_purchase_scd1
   :     +- Relation spark_catalog.default.silver_purchase_scd1[transaction_sid#10793L,add_supplements#10794,is_member#10795,member_discount#10796,price#10797,product_id#10798,quantity#10799L,supplement_price#10800,total_purchase#10801,transaction_i

In [ ]:
# spark.sql("select * from gold_fact_purchase").show(5)
spark.sql("select * from gold_fact_inventory").show(5)

+-------------+--------------------+--------------+---------+----------+--------------+--------------------+--------------------+--------------------+-----------+
|inventory_sid|          event_time|existing_level|new_level|product_id|stock_quantity|       meta_hashdiff|   meta_ingestion_ts|   meta_last_updated|product_sid|
+-------------+--------------------+--------------+---------+----------+--------------+--------------------+--------------------+--------------------+-----------+
|            0|2024-02-17 19:28:...|            34|       44|      SC04|            10|0d93648fbc91fd6f5...|2024-02-20 20:14:...|2024-02-20 20:17:...|34359738374|
|            1|2024-02-17 19:28:...|            49|       59|      SF07|            10|6990e55cf2125fc32...|2024-02-20 20:14:...|2024-02-20 20:17:...|25769803779|
|            2|2024-02-17 19:28:...|            43|       53|      SC04|            10|add0004a8cd62da10...|2024-02-20 20:14:...|2024-02-20 20:17:...|34359738374|
|            3|2024-02